In [2]:
import requests
import pandas as pd

In [59]:
#Rango de fecha de datos: 1973-01-01  ->  2024-07-02

params = {
    'dataset': 'daily-summaries',
    'startDate': '1974-01-01',
    'endDate': '2024-07-02',
    'stations': 'AR000087418',  #Mendoza Aereo
    'format': 'json',
    'includeStationName': 'true',
}


url=f"https://www.ncei.noaa.gov/access/services/data/v1"


In [60]:
response=requests.get(url,params=params)


In [61]:
if response.status_code==200:
    info=response.json()
else:
    print(f"Estatus data 1:-> {response.status_code}")

In [62]:

data=pd.DataFrame(info)

#Por si quieren generar la data en csv
#data.to_csv('datos_mendoza_air')